# Text Pre Processor - Experimento

Este é um componente que utiliza a biblioteca [nltk](https://www.nltk.org/) e [ftfy](https://pypi.org/project/ftfy/) e [regex](https://docs.python.org/3/library/re.html) para pré processar textos que entrrão em outros componentes.

## Declaração de parâmetros e hiperparâmetros

Declare parâmetros com o botão  na barra de ferramentas.<br>
A variável `dataset` possui o caminho para leitura do arquivos importados na tarefa de "Upload de dados".<br>
Você também pode importar arquivos com o botão  na barra de ferramentas.

In [ ]:
# parâmetros
dataset = "/tmp/data/imdb.csv" #@param {type:"string"}
target = "review" #@param {type:"string", label:"Atributo alvo", description:"Seu modelo será treinado para prever os valores do alvo."}
language = "Português" #@param ["Português", "Inglês"] {type:"string", label:"Linguagem", description:"Linguagem da qual os stopwords pertencem. Deve ser a mesma utilizada no dataset."}

# preprocessamento
case = "Caixa baixa" #@param ["Caixa baixa","Caixa alta","Não Aplicar"] {type:"string",label:"Aplicação de casing", description:"Caixa baixa, caixa alta ou não aplicação de caixa"}
remove_stop_words = True #@param {type:"boolean",label:"Remoção de Stop Words", description:"Remoção de palavras, conjunções, artigos e outros"}
remove_top_words = True #@param {type:"boolean",label:"Remoção de Top Words", description:"Remoção da porcentagem de palavras mais frequentes no texto"}
top_words_percentage = 0.01 #@param {type:"number",label:"Porcentagem de Top Words", description:"Porcentagem das palavras mais frequentes no texto"}
stemming = False #@param {type:"boolean",label:"Stemming"}
lemmatization = True #@param {type:"boolean",label:"Lemmatization"}
remove_punctuation = True #@param {type:"boolean",label:"Remoção de pontuação"}
remove_line_breaks = True #@param {type:"boolean",label:"Remoção de quebras de lina",description:"Remoção de quebras de linha por \n e \r"}
remove_accents = True #@param {type:"boolean",label:"Remoção de acentos"}
remove_html = True #@param {type:"boolean",label:"Remoção de HTML"}
remove_css = True #@param {type:"boolean",label:"Remoção de CSS"}

## Traduz parâmetros

In [ ]:
language = "portuguese" if language == "Português" else "english"

if case == "Caixa baixa":
    case = "Lower"
elif case == "Caixa alta":
    case = "Upper"
else:
    case = "NotApply"

## Acesso ao conjunto de dados

O conjunto de dados utilizado nesta etapa será o mesmo carregado através da plataforma.<br>
O tipo da variável retornada depende do arquivo de origem:
- [pandas.DataFrame](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html) para CSV e compressed CSV: .csv .csv.zip .csv.gz .csv.bz2 .csv.xz
- [Binary IO stream](https://docs.python.org/3/library/io.html#binary-i-o) para outros tipos de arquivo: .jpg .wav .zip .h5 .parquet etc

In [ ]:
import pandas as pd

df = pd.read_csv(dataset)

columns = df.columns

## Remoção de linhas com valores faltantes no atributo alvo

Caso haja linhas em que o atributo alvo contenha valores faltantes, é feita a remoção dos casos faltantes.

In [ ]:
import numpy as np

df.dropna(subset=[target], inplace=True)

X = df[target].to_numpy()

## Chamada da classe de Pré-Processamento

In [ ]:
preprocessing_tasks =   {"case": case,
                         "remove_stop_words": remove_stop_words,
                         "remove_top_words": remove_top_words,
                         "top_words_percentage": top_words_percentage,
                         "stemming": stemming,
                         "lemmatization": lemmatization,
                         "remove_punctuation": remove_punctuation,
                         "remove_line_breaks": remove_line_breaks,
                         "remove_accents": remove_accents,
                         "remove_html": remove_html,
                         "remove_css": remove_css}

model_parameters = {'language': language}

In [ ]:
from pre_processor import Preprocessor

processor = Preprocessor(preprocessing_tasks, model_parameters)
text = processor.preprocess(X)

## Salva alterações no conjunto de dados

O conjunto de dados será salvo (e sobrescrito com as respectivas mudanças) localmente, no container da experimentação, utilizando a função `pandas.DataFrame.to_csv`.<br>

In [ ]:
df[target] = text

df.to_csv(dataset, index=False)

## Salva resultados da tarefa 

A plataforma guarda o conteúdo de `/tmp/data/` para as tarefas subsequentes.

In [ ]:
from joblib import dump

artifacts = {
    "preprocessing_tasks": preprocessing_tasks,
    "model_parameters": model_parameters,
    "columns": columns,
    "target": target
}

dump(artifacts, "/tmp/data/preprocessor.joblib")